Here, I will just quickly prove that the idea of vectorizing MNIST and then just classifying with cosine similarity doe not work.

In [2]:
# imports
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import datasets, model_selection
mnist = datasets.fetch_mldata('MNIST original')

data, target = mnist.data, mnist.target

Test how reshape works

In [3]:
resh = np.array([[4, 4, 1, 4],
                [4, 12, 4, 1]])
resh.reshape(1, 8)

array([[ 4,  4,  1,  4,  4, 12,  4,  1]])

Testing cosine similarity, note how it takes one argument and compares the vectors within the matrix you pass in.

In [4]:
# cosine similarity function
v1 = np.array([3, 5, 1, 0, 9, 9])
v2 = np.array([45, 1, 3, 2, 9, 9])

V = []
V.append(v1)
V.append(v2)
V = np.array(V)
V

array([[ 3,  5,  1,  0,  9,  9],
       [45,  1,  3,  2,  9,  9]])

In [5]:
cosine_similarity(V)[0][1]


0.46318736602106464

Messing with MNIST

In [51]:
data.shape

(70000, 784)

In [61]:
np.max(target)

9.0

In [62]:
np.min(target)

0.0

In [63]:
target.shape

(70000,)

In [71]:
target[18623]

3.0

In [65]:
target[0]

0.0

In [66]:
target[1]

0.0

In [70]:
target[50000]

8.0

In [68]:
# see number of unique labels
unique, counts = np.unique(target, return_counts=True)
dict(zip(unique, counts))

{0.0: 6903,
 1.0: 7877,
 2.0: 6990,
 3.0: 7141,
 4.0: 6824,
 5.0: 6313,
 6.0: 6876,
 7.0: 7293,
 8.0: 6825,
 9.0: 6958}

Now do cosine similarity analysis

In [101]:
target[55000]

9.0

In [6]:
# some rnadom indices
indices = [0, 5923, 12665, 18623, 24754, 30596, 36017, 41935, 48200, 54051, 10000, 18000, 22000, 25000, 35000, 38000, 43000, 49000, 55000, 10, 15, 16, 2, 3, 4, 5, 1000]
for i in indices:
    print(target[i])

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [7]:
# store distance values
distance = []
for i in indices:
    ls = []
    ls.append(data[0])
    ls.append(data[i])
    ls = np.array(ls)
    
    # cosine similarity
    cosim = cosine_similarity(ls)[0][1]
    distance.append(cosim)
    
distance

[0.9999999999999998,
 0.2858301903631163,
 0.4497013961534748,
 0.54425142574231,
 0.21570933561782463,
 0.5521001886778968,
 0.5111312023242616,
 0.4791215007284332,
 0.4680902177505057,
 0.41086014904236257,
 0.2704654477897025,
 0.28533822985096535,
 0.3509746939846747,
 0.23026947262329941,
 0.27783015040030595,
 0.28705812477635434,
 0.4089752455132159,
 0.5630656252149693,
 0.36238219732757243,
 0.760059532281908,
 0.7441768322363791,
 0.8865646705054383,
 0.6394367263909786,
 0.78084024048646,
 0.7655111807364987,
 0.6602185524970919,
 0.40968392788021063]

Holy shit.  I never expected that to actually work lol.

In [86]:
data.shape

(70000, 784)

In [85]:
data.T.shape

(784, 70000)

Well that is pretty funny that it sorta works.  I will just try to build a basic version that takes the top most similar values then does weighted voting or something simple like that to determine what the actual number is.

Below is an algorithm to return the top n biggest values of an array.

In [27]:
import heapq
distance = np.array(distance)
heapq.nlargest(5, range(len(distance)), distance.take)[1]

17246

In [9]:
vsdf = np.arange(0, 20000)
heapq.nlargest(5, range(len(vsdf)), vsdf.take)

[19999, 19998, 19997, 19996, 19995]

So it is good that those algorithms are really fast

In [ ]:
# take top n
def vote(arr, n):
    """arr: array whose values will be checked
    n: number of top values you want to return
    return the determined number"""

Just some basic tests to see if generally works

In [23]:
target[54051]

9.0

In [24]:
%%time
distance = []
for i in range(0, len(target)):
    ls = []
    ls.append(data[12665])
    ls.append(data[i])
    ls = np.array(ls)
    
    # cosine similarity
    cosim = cosine_similarity(ls)[0][1]
    distance.append(cosim)
    
distance = np.array(distance)
distance.shape

CPU times: user 6.4 s, sys: 3.94 ms, total: 6.41 s
Wall time: 6.41 s


In [28]:
top = heapq.nlargest(10, range(len(distance)), distance.take)
top[1]

17246

In [26]:
for i in top:
    print(target[i])

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0


In [130]:
indices = [0, 5923, 12665, 18623, 24754, 30596, 36017, 41935, 48200, 54051, 10000, 18000, 22000, 25000, 35000, 38000, 43000, 49000, 55000]
for i in indices:
    print(target[i])

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0


Note how everything except for 5, works really well.